In [1]:
import os
from sklearn.model_selection import train_test_split

%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

seed=42

In [2]:
lr = 1e-3
batch_size = 32
epochs = 10

10

In [3]:
base_data_dir = '../data'

input_imgs = sorted([os.path.join(base_data_dir, 'unshiny', fname) for fname in os.listdir(os.path.join(base_data_dir, 'unshiny')) if fname.endswith('.png')])
label_imgs = sorted([os.path.join(base_data_dir, 'shiny', fname) for fname in os.listdir(os.path.join(base_data_dir, 'shiny')) if fname.endswith('.png')])

assert len(input_imgs) == len(label_imgs)
assert all(os.path.basename(input_img) == os.path.basename(label_img) for input_img, label_img in zip(input_imgs, label_imgs))

print(f'Found {len(input_imgs)} images')

Found 12239 images


In [4]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_imgs, label_imgs, test_size=0.2, random_state=seed)

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
class AutoEncoderDataset(Dataset):
    def __init__(self, input_imgs, label_imgs, transforms=None):
        assert len(input_imgs) == len(label_imgs)
        # not supporting LA mode images
        self.input_imgs = [img for img in input_imgs if Image.open(img).mode != 'LA']
        self.label_imgs = [img for img in label_imgs if Image.open(img).mode != 'LA']
        self.transforms = transforms

        assert len(self.input_imgs) == len(self.label_imgs)
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.item()

        input_img = Image.open(self.input_imgs[idx]).convert('RGB')
        label_img = Image.open(self.label_imgs[idx]).convert('RGB')
        return self.transforms(input_img), self.transforms(label_img)
    def __len__(self):
        return len(self.input_imgs)
        pa

In [7]:
ideal_imsize = (80, 80)
transforms = transforms.Compose([
    transforms.Resize(ideal_imsize),
    transforms.ToTensor()
])

train_ds = AutoEncoderDataset(train_inputs, train_labels, transforms)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

# TODO: add validation dataset

test_ds = AutoEncoderDataset(test_inputs, test_labels, transforms)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

AssertionError: 

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, default_activation=nn.ReLU(), latent_dim=512):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, 5, stride=2),
            default_activation,
            nn.Conv2d(32, 64, 3, stride=2),
            default_activation,
            nn.Conv2d(64, 128, 3, stride=2),
            default_activation
        )
        # self.latent = nn.Sequential(
        #     nn.Flatten(),
        #     nn.Linear(8192, latent_dim),
        #     default_activation,
        #     nn.Linear(latent_dim, 8192),
        # )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2),
            default_activation,
            nn.ConvTranspose2d(64, 32, 3, stride=2),
            default_activation,
            nn.ConvTranspose2d(32, out_channels, 5, stride=2),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        # x = self.latent(x)
        x = self.decoder(x)
        return x

In [ ]:
model = AutoEncoder().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

MSELoss()

In [ ]:
import time
model.train()

start_time = time.time()
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dl):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_dl)}, Time: {time.time() - start_time:.2f}s')

print(f'Training finished, Time: {time.time() - start_time:.2f}s')
print(f'Average time per epoch: {(time.time() - start_time) / epochs:.2f}s')

/home/lucaskeller/anaconda3/envs/ai2024/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 3, 80, 80])) that is different to the input size (torch.Size([32, 3, 73, 73])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (73) must match the size of tensor b (80) at non-singleton dimension 3